In [ ]:
from PIL import Image
import os
import numpy as np
from matplotlib import pyplot as plt
import yaml
import pickle
import pandas as pd

In [ ]:
for _, dir, files in os.walk('.'):
    break

In [ ]:
dir = [d for d in dir if 'output_' in d]
conf_file = [d for d in files if 'yaml' in d and 'gen' not in d][0]

In [ ]:
with open(conf_file, "r") as f:
    config = yaml.safe_load(f)

In [ ]:
for _, _, files in os.walk('./inference_sdxl'):
    break

In [ ]:
dir_files = {}
name_files = set()
for d in dir:
    dir_files[d] = {}
    dir_files[d]['files'] = []
    for _, cd, fl in os.walk(f'./{d}'):
        break
    conf_file = [d for d in fl if 'log' in d ][0]
    with open(f"./{d}/{conf_file}", "r") as f:
        config = yaml.safe_load(f)
    dir_files[d]['config'] = config
    if 'inference' in cd:
        for _, _, files in os.walk(f'./{d}/inference'):
            break
        dir_files[d]['files'] += files
        name_files = name_files.union(set(files))

In [ ]:
import os
os.makedirs(f"./reconciliation", exist_ok=True)

# 100, 80, 40, 20
for n, file_name in enumerate(name_files):
    if '100' not in file_name:
        continue
    f, axarr = plt.subplots(2, (len(dir) + 2) // 2, figsize=(30, 15))
    axarr[0][0].imshow(Image.open('target.jpg'))
    axarr[0][0].set_title("Target")
    axarr[0][0].axis('off')
    for m, d in enumerate(dir):
        
        if file_name in dir_files[d]['files']:
            axarr[(m+1) // ((len(dir) + 2) // 2)][(m+1) % ((len(dir) + 2) // 2)].imshow(Image.open(f'./{d}/inference/{file_name}'))
            if dir_files[d]['config']['use_mask']:
                title = f"mask, p={dir_files[d]['config']['mask_power']}, a={dir_files[d]['config']['mask_alpha']}"
            elif dir_files[d]['config']['use_zone']:
                title = f"zone, a={dir_files[d]['config']['zone_alpha']}"
            else:
                title="OneActor"
            title = f"{d}\n{title}"
            axarr[(m+1) // ((len(dir) + 2) // 2)][(m+1) % ((len(dir) + 2) // 2)].set_title(title)
            axarr[(m+1) // ((len(dir) + 2) // 2)][(m+1) % ((len(dir) + 2) // 2)].axis('off')
    f.suptitle(f'{file_name}', fontsize=14)
    plt.show()
    f.savefig(f'./reconciliation/rec_{file_name}')       

In [ ]:
# load ds dcore
with open(f"metric_ds.pickle", 'rb') as handle:
    ds_score = pickle.load(handle)

In [ ]:
ds_score[(ds_score['exp'] == 'exp_2504211015') & (ds_score['epoch'] == 100)].groupby(['model', 'title']).mean('distance')

In [ ]:
import os
os.makedirs(f"./reconciliation", exist_ok=True)

EPOCH = 100
THEMA="teenager"

dir_short = ['output_2504211225','output_2504211945']
name_files_short = [f for f in name_files if str(EPOCH) in f]


for n, file_name in enumerate(name_files_short):
    f, axarr = plt.subplots(1, (len(dir_short) + 1), figsize=(15, 6))
    image1 = Image.open('target.jpg')
    axarr[0].imshow(image1)
    axarr[0].set_title("Target")
    axarr[0].axis('off')
    for m, d in enumerate(dir_short):
        image2 = Image.open(f'./{d}/inference/{file_name}')
        axarr[m+1].imshow(image2)
        if dir_files[d]['config']['use_mask']:
            title = f"mask, p={dir_files[d]['config']['mask_power']}, a={dir_files[d]['config']['mask_alpha']}"
        elif dir_files[d]['config']['use_zone']:
            title = f"zone, a={dir_files[d]['config']['zone_alpha']}"
        else:
            title="OneActor"
        prompt = ('_'.join(file_name.split(".")[0].split("_")[:-1]))
        score = ds_score[(ds_score['prompt'] == prompt) & (ds_score['epoch'] == EPOCH) & (ds_score['model'] == d)]['dreamsimScore'].values[0]
        clip_score = ds_score[(ds_score['prompt'] == prompt) & (ds_score['epoch'] == EPOCH) & (ds_score['model'] == d)]['CLIPScore'].values[0]
        title = f"{title}\ndrieamsim score: {score:0.3f}\nCLIPScore: {clip_score}"
        axarr[m+1].set_title(title, color="red" if m%2 == 1 else "black")
    plt.suptitle(file_name)
    plt.show()
    f.savefig(f'./reconciliation/{THEMA}_rec_OA_vs_Mask')
print("score = 1 for identical images")

In [ ]:
ds_score[
    (ds_score['thema'] == 'adventurer_with_tousled_hair_comic_book') & (ds_score['epoch'] == 100) & (ds_score['title'].isin(['OneActor', 'mask, p=0.5, a=0.5']))
    ].groupby('title').agg({'dreamsimScore':'mean', 'CLIPScore':'mean'})

In [ ]:
!cp reconciliation.ipynb /workspace/Diploma/notebooks